In [1]:
import numpy as np
import matplotlib.pyplot as plt
    

In [2]:
def convert_rgb(org_color):   
    """org_color is type string: '174.199.232'
    # Original values are scaled 1-255
    # rgb_color is type tuple
    # Final color is scaled 0-1: (0.643,0.416,0.894)"""
    
    str_vec = org_color.strip().split('.')#type list - of strings
    num_vec = [float(s) for s in str_vec]#type list - of floats
    rgb_scale = [(n/255) for n in num_vec]#type list - of scaled floats
    
    # Note: Tried a regular expression...
    # It doesn't work - e.g.,:  t2 = [t[i] for i in t]
    # TypeError: tuple indices must be integers, not float
    rgb_tuple = (rgb_scale[0],rgb_scale[1],rgb_scale[2])
    return rgb_tuple
    
    

In [3]:
def tableau10():
    """ returns a dict with color names as keys and converted rgb tuples as values
    #http://tableaufriction.blogspot.ro/2012/11/finally-you-can-use-tableau-data-colors.html
    #References convert_rgb()"""

    tableau={}#dict
    color_key = [
    'dark_grey','med_grey','med2_grey','med3_grey','med4_grey','light_grey',
    'blue','orange','green','red','purple',
    'turq','puke','med_puke','light_puke',
    'brown','lav',
    'dark_red','bright_red','fushia_red','salmon','blue_red','light_red',
    'cool_blue','purp_blue','med_blue','med2_blue','med3_blue',
    'lav_blue','sky_blue','light_cool_blue']
    color_str = [
    '50.50.50','75.75.75','100.100.100','125.125.125','150.150.150','200.200.200',
    '31.119.180','255.127.14','44.160.44','214.39.40','148.103.189',
    '23.190.207','188.189.34','205.204.93','219.219.141',
    '140.86.75','227.119.194',
    '177.3.24','240.39.32','189.10.54','242.108.100','177.3.74','254.69.70',
    '44.105.176','144.158.206','107.163.214','137.183.214','95.158.209',
    '181.200.226','177.213.240','94.155.225']
    
    for idx in range(0,len(color_key)):
        #print(color_str[idx])
        tup = convert_rgb(color_str[idx])
        #tableau[color_key[idx]] = color_str[idx]
        tableau[color_key[idx]] = tup
    
    #For Displaying/debugging
    #print(tableau.items())
    #print(tableau['dark_grey'])#access by color-name
    return tableau

In [4]:
class color1:
    """ Class that defines colors in a palette for a bar graph"""
    t = tableau10()#color settings

    c = [ t['cool_blue'],t['red'] ]#marker color: 0=healthy/control and 1=sick/target groups
    err = [ t['med2_grey'],t['med2_grey'] ]#error bar color
    edge = [ t['med2_grey'] ]
    
    xlb = t['dark_grey']; ylb = t['dark_grey']#labels
    xtk = t['med3_grey']; ytk = t['med3_grey']#ticks
    ytl = t['med3_grey']#ticklines
    leg = t['med3_grey']#legend box
    box = t['med4_grey']
    

In [5]:
def ka_bar(emmeans, sem, p, c, ka_labels):
    """Plot a bar-graph of two groups with multiple DVs"""
    ind = np.arange(len(emmeans[:,1]))#This may throw error if emmeans has only one DV
    width = 0.35
    
    # Instatiate objects
    fig, ax = plt.subplots()
    
    # Plot data as bars
    rects1 = ax.bar(ind, emmeans[:,1], width, color=c.c[1],yerr=sem[:,1],ecolor = c.err[1],edgecolor=c.edge)
    rects2 = ax.bar(ind+width, emmeans[:,0], width, color=c.c[0],yerr=sem[:,0],ecolor=c.err[0], edgecolor=c.edge)
    #print(plt.getp(rects1))# A way to inspect the properties of an artist object
    
    # Format axes labels, ticks, and lines
    plt.rc('axes',ec=c.box)
    ax.set_xlabel(ka_labels.axes[0], fontsize=12, color=c.xlb)
    ax.set_ylabel(ka_labels.axes[1], fontsize=12, color=c.ylb)
    #ax.set_title('FA Means by Group')
    ax.set_xticks(ind+width)
    ax.set_xticklabels(ka_labels.xtick,fontsize=10, color=c.xtk)
    plt.setp([ax.get_yticklabels()],color=c.ytk)
    
    plt.setp([ax.get_xticklines()], visible=False)
    plt.setp([ax.get_yticklines()], color=c.ytl)
    
    # Max & min values for Y-axis
    #plt.ylim([np.min(X), np.max(X)])#good to start with
    plt.ylim([.35, .60])#hard-coded
    
    # Set Legend
    leg = ax.legend( (rects1[0], rects2[0]), (ka_labels.legend[1],ka_labels.legend[0]))
    leg.get_frame().set_edgecolor(c.box)
    for text in leg.get_texts():
        plt.setp(text, color=c.leg)
    
    # Add p-values as a text annotation
    for i in range(0, len(emmeans[:,1])):  
        # These if statements just help me tailor the placement of these p-values
        # otherwise, they can be commented out
        if i==0: ypad = 2.1
        else: ypad = 3
        xpad = 0
        #xpad = .3*width
        yloc_p = max(emmeans[i,0],emmeans[i,1]) + (ypad*max(sem[i,0],sem[i,1]))
        ax.text(ind[i]+xpad, yloc_p, p[i], fontsize=8.5, color=c.xtk)
    
    ax.margins(0.05)
    plt.show()
    return fig

In [6]:
class get_ptsd_emmeans():
    """ PTSD EMMEANS for All tracts w/ age, hispanic, college, medical-exception1 in n=58 (27 CTRL,31 PTSD+ )
    # CTRL first, then PTSD means - taken from SPSS ANOVA, EMMEANS(ptsd_status)"""
    emmeans = np.array([ [.532,.558],[.489,.504],[.469,.491],[.461,.479],[.421, .440],[.405,.404],[0,0],[.463,.479] ])#CTRL, PTSD on ACR
    sem = np.array([ [.010,.009],[.006,.005],[.008,.008],[.006,.006],[.006,.005],[.010,.009],[0,0],[.005,.005] ])
    pvalue = ([ '.060†','.072†','.061†','.043*','.023*','.935','','.028*' ])    
    
    if len(emmeans) != len(pvalue) or len(emmeans) != len(sem) or len(sem) != len(pvalue):
        print('\nCheck lengths of emmeans, sem, and p-values: At least one is not equivalent\n')
    
    class labels():
        xtick = ['CGC','SLF','IFOF','SFOF','ACR','UF','','All Tracts']
        axes = ['White Matter Tracts','Mean Bilateral Fractional Anisotrophy']
        legend = ['CTRL','PTSD']

In [7]:
class get_hispanic_emmeans():
    """  Hispanic EMMEANS for 'SLF','IFOF','ACR', controlling for age, college, medical_exception1, ptsd in n=58
    #  Non-Hispanic, then Yes/Hispanic - same order as SPSS - ANOVA, EMMEANS(hispanic)"""
    emmeans = np.array([ [.506,.489],[.494,.468],[.438,.423] ])#CTRL, PTSD on ACR
    sem = np.array([ [.006,.005],[.008,.008],[.006,.005] ])
    # p-values: .031, .031, .052
    
    class labels():
        xtick = ['SLF','IFOF','ACR']
        axes = ['White Matter Tracts','Mean Bilateral Fractional Anisotrophy']
        legend = ['Non-Hispanic','Hispanic']
        
####-----------------------------------------------------------------------###    

In [8]:
def main():
    
    # Instatiate Class Objects
    palette = color1()   
    ptsd = get_ptsd_emmeans()
    #hisp = get_hispanic_emmeans()
    
    # Make & Save Bargraph of PTSD Group Differences
    folder = '/Users/kaschbacher/Desktop/Python/PTSD/'
    barfig = ka_bar(ptsd.emmeans, ptsd.sem, ptsd.pvalue, palette, ptsd.labels)
    barfig.savefig((folder+'Figures/'+'Group diff all tracts and pvalues.eps'))
    
if __name__=="__main__":
    main()

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 4: ordinal not in range(128)